In [169]:
path = 'data/emails.txt'
with open(path, 'rb') as f:
    text = f.read().decode('utf-8', errors='ignore')

In [170]:
import re

def remove_html_tags(text_with_html):
    # Define a regular expression pattern to match HTML tags
    html_tag_pattern = re.compile('<.*?>')

    # Use the `sub` method to replace all matched HTML tags with an empty string
    text_without_html = re.sub(html_tag_pattern, '', text_with_html)

    return text_without_html

text = remove_html_tags(text)

In [171]:
import regex

def cleanup(dirty_text):

    # Define a regular expression pattern to match nested square brackets
    square_brackets_pattern = regex.compile(r'\[(?:[^[\]]*|(?R))*\]')
    dirty_text = regex.sub(square_brackets_pattern, '', dirty_text)

    # Define a regular expression pattern to match text inside square brackets
    square_brackets_pattern = re.compile(r'\[.*?\]')
    dirty_text = re.sub(square_brackets_pattern, '', dirty_text)

    # Weird character
    dirty_text = dirty_text.replace("=E2=80=99", "`")

    # Define a regular expression pattern to match '=E2=80=' followed by a number
    e2_80_sequence_pattern = re.compile(r'=E2=80=\d+')
    dirty_text = re.sub(e2_80_sequence_pattern, '', dirty_text)

    # Define a regular expression pattern to match sequences starting with '=' and ending with a number
    equals_sequence_pattern = re.compile(r'=([A-F0-9]{2}=)+\d+')
    dirty_text = re.sub(equals_sequence_pattern, '', dirty_text)

    # Define a regular expression pattern to match '=' followed by any non-letter characters
    equals_non_letters_pattern = re.compile('=([^a-zA-Z]+)')
    dirty_text = re.sub(equals_non_letters_pattern, '', dirty_text)

    clean_text = dirty_text.replace("=E2", "`")

    return clean_text

def remove_minute_read(text):
    # Define a regular expression pattern to match the sequence
    minute_read_pattern = re.compile(r'\(?\d*\s*MINUTE\s*READ\)?', re.IGNORECASE)

    # Use the `sub` method to replace all matched sequences with an empty string
    clean_text = re.sub(minute_read_pattern, '', text)

    return clean_text

text = cleanup(text)
text = remove_minute_read(text)

In [172]:
arguments = [
    "BIG TECH & STARTUPS",
    "SCIENCE & FUTURISTIC TECHNOLOGY",
    "PROGRAMMING, DESIGN & DATA SCIENCE",
    "MISCELLANEOUS"
]

def extract_sections(text, start_section, end_section):
    # Define the regular expression pattern to match the desired sections
    pattern = re.compile(r'(?i){}(.*?)(?={})'.format(re.escape(start_section), re.escape(end_section)), re.MULTILINE | re.DOTALL)
    sections = re.findall(pattern, text)

    return sections

In [173]:
text_by_argument = {}

# Iterate through the sections_to_extract list using indices
for i in range(len(arguments) - 1):
    argument = arguments[i]
    next_argument = arguments[i + 1]

    text_by_argument[argument] = extract_sections(text, argument, next_argument)

for section, content in text_by_argument.items():
    print(f"{section}: {len(content)} sections extracted")

BIG TECH & STARTUPS: 213 sections extracted
SCIENCE & FUTURISTIC TECHNOLOGY: 212 sections extracted
PROGRAMMING, DESIGN & DATA SCIENCE: 240 sections extracted


In [174]:
def print_first_10_sections(sections):
    for i, section in enumerate(sections[:10], start=1):
        print(section)

#print the first 10 extracted section for the BIG TECH & STARTUPS argument
print_first_10_sections(text_by_argument[arguments[0]])



BIG NEWS FROM MICROSOFT CTO ANDREAS BRAUN 



Microsoft CTO Andreas Braun announced that GPT-4 will be released as
early as next week. The new model will be multimodal and will have
video capabilities. The announcement was made during the 'AI in Focus
- Digital Kickoff' hybrid event held on March 9 whereMicrosoft
Germany employees presented LLMs like the GPT series.GOOGLE DUSTS OFF THE FAILED GOOGLE+ PLAYBOOK TO FIGHT CHATGPT 



Google was threatened by Facebook's rise to success in 2011. This
prompted thecompany to issue a decree to its employees to build
social features intoeverything, with their bonuses tied to Google's
social success. The decree resulted in ham-fisted social integrations
across Google that users despised, such as YouTube comments being tied
to Google+ and the real name policy. Just like with the company's
social panic, its current ChatGPT panic may see employees' ratings and
reviews for promotions influenced by their ability to integrate
artificial intelligence 